In [10]:
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt 
from sklearn.neural_network import MLPClassifier
import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import texthero as hero

## I. Jeu de donnée Kaagle

In [11]:
df = pd.read_csv("Emotion_final.csv")
df .describe()

,Text,Emotion
count,21459,21459
unique,21405,6
top,i feel so tortured by it,happy
freq,2,7029


In [12]:
df_x=df.iloc[:,1:]
df_y=df.iloc[:,0]

In [15]:
df

,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
21454,Melissa stared at her friend in dism,fear
21455,Successive state elections have seen the gover...,fear
21456,Vincent was irritated but not dismay,fear
21457,Kendall-Hume turned back to face the dismayed ...,fear


In [18]:
df['clean_text'] = hero.clean(df.Text)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(df.clean_text, df.Emotion, train_size=0.85, random_state=2, shuffle=True)
X_test, X_valid, y_test, y_valid = train_test_split(X_train, y_train, train_size=0.5, random_state=2, shuffle=True)

In [21]:
df

,Text,Emotion,clean_text
0,i didnt feel humiliated,sadness,didnt feel humiliated
1,i can go from feeling so hopeless to so damned...,sadness,go feeling hopeless damned hopeful around some...
2,im grabbing a minute to post i feel greedy wrong,anger,im grabbing minute post feel greedy wrong
3,i am ever feeling nostalgic about the fireplac...,love,ever feeling nostalgic fireplace know still pr...
4,i am feeling grouchy,anger,feeling grouchy
...,...,...,...
21454,Melissa stared at her friend in dism,fear,melissa stared friend dism
21455,Successive state elections have seen the gover...,fear,successive state elections seen governing part...
21456,Vincent was irritated but not dismay,fear,vincent irritated dismay
21457,Kendall-Hume turned back to face the dismayed ...,fear,kendall hume turned back face dismayed coup


In [ ]:
per = per

NameError: name 'Perceptron' is not defined

## II. Jeu de donnée data world 

In [ ]:
df = pd.read_csv("text_emotion.csv")
df .describe()

,tweet_id
count,4.000000e+04
mean,1.845184e+09
std,1.188579e+08
min,1.693956e+09
25%,1.751431e+09
50%,1.855443e+09
75%,1.962781e+09
max,1.966441e+09
